In [1]:
import time
import json
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Reshape, Conv1D, Subtract, Activation, Flatten, Lambda, Add, Multiply, Bidirectional, Dense, BatchNormalization, SpatialDropout1D, LSTM
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError, BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model


import sys
sys.path.append("../")
from config import APP_CONFIG, JACK_KELLY_INPUT_CONFIG, DEFAULT_STEP_SIZE, FORWARD_FILLING_WINDOW

from __models__ import *
from util import *

In [2]:
# disable tensor INFO
import logging
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'  # filter tensorflow INFO
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [3]:
# create weigth folder
model_folder = './benchmark/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [4]:
list_devices = ["GPU:0", "GPU:1", "GPU:2", "GPU:3"]
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [5]:
# experiment setup
epochs = 50
list_models = [JKDAE, JKLSTM, S2S, BiTCNResidual, FullyConvolutionalNetwork]
off_on_ratio=4 # 25% data contain activation
split_ratio = (0.9,0.05,0.05) # ratio<-train:val:test 
real_data_only=True

In [6]:
# list to store result in 
list_appliance = []
list_model_name = []
list_metric = []
list_evaluation = []

In [7]:
# 1. Evaluate regular model
x_name=['agg_power']
y_name=['app_power']
for appliance in APP_CONFIG.keys():
    [X], [Y] = load_data(appliance, x_name, y_name, real_data_only=real_data_only, off_on_ratio=off_on_ratio)
    scaler = STDScaler(X)
    X = scaler.transfer(X)
    scaler_ = STDScaler(Y)
    Y = scaler_.transfer(Y)
    
    train_x, val_x, test_x = split_data(X, split_ratio)
    train_y, val_y, test_y = split_data(Y, split_ratio)
    
    # train_generator = S2SDataGenerator(train_x, train_y)
    # val_generator = S2SDataGenerator(val_x, val_y)
    test_generator = S2SDataGenerator_beta([test_x], [test_y])
    for model_prototype in list_models:
        with tf.distribute.MirroredStrategy(list_devices).scope():
            sequence_length = JACK_KELLY_INPUT_CONFIG[appliance]//DEFAULT_STEP_SIZE
            model_class = model_prototype(sequence_length, sequence_length, appliance, 'reg', epochs)
            model_class.load_weights()

            test_loss, test_mae = model_class.model.evaluate_generator(test_generator)
            print('{:40s} MAE: {:.3f}'.format(model_class.name, test_mae*scaler_.std))
            
            list_appliance.append(appliance)
            list_model_name.append('_'.join(model_class.name.split('_')[1: -2]))
            list_metric.append('mae')
            list_evaluation.append(test_mae*scaler_.std)

Loading Data for kettle
Time elapse: 25.12s
kettle_jkdae_128to128_reg                MAE: 50.450
kettle_jklstm_128to128_reg               MAE: 12.016
kettle_s2s_128to128_reg                  MAE: 28.536
kettle_bi_tcn_residual_128to128_reg      MAE: 35.206
kettle_fcn_128to128_reg                  MAE: 11.592
Loading Data for fridge
Time elapse: 71.76s
fridge_jkdae_512to512_reg                MAE: 29.845
fridge_jklstm_512to512_reg               MAE: 2.856
fridge_s2s_512to512_reg                  MAE: 8.533
fridge_bi_tcn_residual_512to512_reg      MAE: 13.821
fridge_fcn_512to512_reg                  MAE: 4.401
Loading Data for microwave
Time elapse: 40.52s
microwave_jkdae_288to288_reg             MAE: 23.975
microwave_jklstm_288to288_reg            MAE: 5.133
microwave_s2s_288to288_reg               MAE: 22.150
microwave_bi_tcn_residual_288to288_reg   MAE: 24.806
microwave_fcn_288to288_reg               MAE: 5.745
Loading Data for dishwasher
Time elapse: 14.85s
dishwasher_jkdae_1365to1365

In [8]:
# 2. Evaluate classification model
x_name=['agg_power']
y_name=['activate']


for appliance in APP_CONFIG.keys():
    # load data
    [X], [Y] = load_data(appliance, x_name, y_name, real_data_only=real_data_only, off_on_ratio=off_on_ratio)
    scaler = STDScaler(X)
    X = scaler.transfer(X)
    
    train_x, val_x, test_x = split_data(X, split_ratio)
    train_y, val_y, test_y = split_data(Y, split_ratio)
    
    # train_generator = S2SDataGenerator(train_x, train_y)
    # val_generator = S2SDataGenerator(val_x, val_y)
    test_generator = S2SDataGenerator_beta([test_x], [test_y])

    for model_prototype in list_models:
        with tf.distribute.MirroredStrategy(list_devices).scope():
            sequence_length = JACK_KELLY_INPUT_CONFIG[appliance]//DEFAULT_STEP_SIZE
            model_class = model_prototype(sequence_length, sequence_length, appliance, 'clf', epochs)
            model_class.load_weights()

            test_loss, test_acc = model_class.model.evaluate_generator(test_generator)
            print('{:40s} Acc: {:.3f}%'.format(model_class.name, test_acc*100))

            list_appliance.append(appliance)
            list_model_name.append('_'.join(model_class.name.split('_')[1: -2]))
            list_metric.append('acc')
            list_evaluation.append(test_acc)

Loading Data for kettle
Time elapse: 25.30s
kettle_jkdae_128to128_clf                Acc: 98.393%
kettle_jklstm_128to128_clf               Acc: 99.361%
kettle_s2s_128to128_clf                  Acc: 97.827%
kettle_bi_tcn_residual_128to128_clf      Acc: 96.009%
kettle_fcn_128to128_clf                  Acc: 99.325%
Loading Data for fridge
Time elapse: 68.56s
fridge_jkdae_512to512_clf                Acc: 71.645%
fridge_jklstm_512to512_clf               Acc: 71.622%
fridge_s2s_512to512_clf                  Acc: 71.191%
fridge_bi_tcn_residual_512to512_clf      Acc: 72.146%
fridge_fcn_512to512_clf                  Acc: 76.510%
Loading Data for microwave
Time elapse: 40.39s
microwave_jkdae_288to288_clf             Acc: 97.899%
microwave_jklstm_288to288_clf            Acc: 97.167%
microwave_s2s_288to288_clf               Acc: 97.220%
microwave_bi_tcn_residual_288to288_clf   Acc: 97.167%
microwave_fcn_288to288_clf               Acc: 98.472%
Loading Data for dishwasher
Time elapse: 14.98s
dishwas

In [9]:
def clip_by_clf(predict_clf, agg_power, threshold=0.5):
    is_on = predict_clf > 0.5
    is_off = np.invert(is_on)
    
    if np.sum(is_off) > 0:
        background = np.sum(is_off * agg_power) / np.sum(is_off)
        return (agg_power - (is_on*background)) * is_on
    return agg_power * is_on

In [10]:
# 2.1 Evaluate clip with classification
x_name=['agg_power']
y_name=['app_power', 'activate']

for appliance in APP_CONFIG.keys():
    # load data
    [X], [Y, labels] = load_data(appliance, x_name, y_name, real_data_only=real_data_only, off_on_ratio=off_on_ratio)
    scaler = STDScaler(X)
    X = scaler.transfer(X)

    train_x, val_x, test_x = split_data(X, split_ratio)
    train_y, val_y, test_y = split_data(Y, split_ratio)
    _, _, label_y = split_data(labels, split_ratio)
    
    test_generator = S2SDataGenerator_beta([test_x], [label_y], shuffle=False)

    for model_prototype in list_models:
        with tf.distribute.MirroredStrategy(list_devices).scope():
            sequence_length = JACK_KELLY_INPUT_CONFIG[appliance]//DEFAULT_STEP_SIZE
            model_class = model_prototype(sequence_length, sequence_length, appliance, 'clf', epochs)
            model_class.load_weights()
            
            # multi gpu must use batch, otherwise yield error
            predictions = model_class.model.predict_generator(test_generator)
            
            # predictions = model_class.model.predict(test_x, batch_size=32)

            test_x_unscaled = scaler.transfer_back(test_x)
            pred_y = []
            for i in range(predictions.shape[0]):
                predict_clf = predictions[i]
                agg_power = test_x_unscaled[i]

                pred_y.append(clip_by_clf(predict_clf, agg_power))
            pred_y = np.array(pred_y)
            
            # batch ignore tail
            test_mae = np.sum(np.abs(pred_y - test_y[:pred_y.shape[0]])) / (pred_y.shape[0]*pred_y.shape[1])
            print('{:40s} MAE: {:.3f}'.format(model_class.name, test_mae))
            
            list_appliance.append(appliance)
            list_model_name.append('_'.join(model_class.name.split('_')[1: -2]) +  '_clip')
            list_metric.append('mae')
            list_evaluation.append(test_mae)
            
            
    #         test_loss, test_acc = model_class.model.evaluate_generator(test_generator)
    #         print('{:40s} Acc: {:.3f}%'.format(model_class.name, test_acc*100))

    #         list_appliance.append(appliance)
    #         list_model_name.append('_'.join(model_class.name.split('_')[1: -2]))
    #         list_metric.append('acc')
    #         list_evaluation.append(test_acc)

Loading Data for kettle
Time elapse: 33.02s
kettle_jkdae_128to128_clf                MAE: 48.078
kettle_jklstm_128to128_clf               MAE: 29.251
kettle_s2s_128to128_clf                  MAE: 60.276
kettle_bi_tcn_residual_128to128_clf      MAE: 95.629
kettle_fcn_128to128_clf                  MAE: 29.807
Loading Data for fridge
Time elapse: 95.25s
fridge_jkdae_512to512_clf                MAE: 181.855
fridge_jklstm_512to512_clf               MAE: 295.941
fridge_s2s_512to512_clf                  MAE: 162.931
fridge_bi_tcn_residual_512to512_clf      MAE: 272.944
fridge_fcn_512to512_clf                  MAE: 233.633
Loading Data for microwave
Time elapse: 53.94s
microwave_jkdae_288to288_clf             MAE: 32.032
microwave_jklstm_288to288_clf            MAE: 31.014
microwave_s2s_288to288_clf               MAE: 32.750
microwave_bi_tcn_residual_288to288_clf   MAE: 31.014
microwave_fcn_288to288_clf               MAE: 30.187
Loading Data for dishwasher
Time elapse: 20.22s
dishwasher_jkdae_

In [11]:
# 3. Evaluate model with std block
x_name=['agg_power']
y_name=['app_power', 'std_3']

for appliance in APP_CONFIG.keys():
    # load data
    [X], [Y, std] = load_data(appliance, x_name, y_name, real_data_only=real_data_only, off_on_ratio=off_on_ratio)

    scaler = STDScaler(X)
    X = scaler.transfer(X)
    scaler_ = STDScaler(Y)
    Y = scaler_.transfer(Y)
    std = std / scaler_.std

    train_x, val_x, test_x = split_data(X, split_ratio)
    train_y, val_y, test_y = split_data(Y, split_ratio)
    train_std, val_std, test_std = split_data(Y, split_ratio)
    
    #train_generator = S2SDataGenerator_beta([train_x], [train_y, train_std])
    #val_generator = S2SDataGenerator_beta([val_x], [val_y, val_std])
    test_generator = S2SDataGenerator_beta([test_x], [test_y, test_std])
                
    for model_prototype in list_models:
        with tf.distribute.MirroredStrategy(list_devices).scope():
            sequence_length = JACK_KELLY_INPUT_CONFIG[appliance]//DEFAULT_STEP_SIZE
            model_class = STD(sequence_length, sequence_length, appliance, 'reg_std', epochs, model_prototype)
            model_class.load_weights()
            
            _, _, _, test_mae, _ = model_class.model.evaluate_generator(test_generator)
            print('{:40s} MAE: {:.3f}'.format(model_class.name, test_mae*scaler_.std))
            
            list_appliance.append(appliance)
            list_model_name.append('_'.join(model_class.name.split('_')[1: -3]) + '_std_block')
            list_metric.append('mae')
            list_evaluation.append(test_mae*scaler_.std)

Loading Data for kettle
Time elapse: 35.70s
kettle_jkdae_128to128_reg_std            MAE: 59.455
kettle_jklstm_128to128_reg_std           MAE: 10.821
kettle_s2s_128to128_reg_std              MAE: 24.794
kettle_bi_tcn_residual_128to128_reg_std  MAE: 33.474
kettle_fcn_128to128_reg_std              MAE: 12.068
Loading Data for fridge
Time elapse: 104.45s
fridge_jkdae_512to512_reg_std            MAE: 43.311
fridge_jklstm_512to512_reg_std           MAE: 42.944
fridge_s2s_512to512_reg_std              MAE: 43.281
fridge_bi_tcn_residual_512to512_reg_std  MAE: 152.949
fridge_fcn_512to512_reg_std              MAE: 43.294
Loading Data for microwave
Time elapse: 57.98s
microwave_jkdae_288to288_reg_std         MAE: 58.143
microwave_jklstm_288to288_reg_std        MAE: 56.359
microwave_s2s_288to288_reg_std           MAE: 20.760
microwave_bi_tcn_residual_288to288_reg_std MAE: 24.734
microwave_fcn_288to288_reg_std           MAE: 6.633
Loading Data for dishwasher
Time elapse: 21.67s
dishwasher_jkdae_13

In [12]:
# 4. Evaluate model with std loss
model_type = 'reg_std_loss'
lst_x_names=['agg_power']
lst_y_names=['app_power']

for appliance in APP_CONFIG.keys():
    # load data
    [X], [Y] = load_data(appliance, lst_x_names, lst_y_names, real_data_only=real_data_only, off_on_ratio=off_on_ratio)
    scaler = STDScaler(X)
    X = scaler.transfer(X)
    if model_type == 'reg_std_loss':
        scaler_ = STDScaler(Y)
        Y = scaler_.transfer(Y)

    train_x, val_x, test_x = split_data(X, split_ratio)
    train_y, val_y, test_y = split_data(Y, split_ratio)

    # train_generator = S2SDataGenerator(train_x, train_y)
    # val_generator = S2SDataGenerator(val_x, val_y)
    test_generator = S2SDataGenerator(test_x, test_y)

    for model_prototype in list_models:
        with tf.distribute.MirroredStrategy(list_devices).scope():
            sequence_length = JACK_KELLY_INPUT_CONFIG[appliance]//DEFAULT_STEP_SIZE
            model_class = model_prototype(sequence_length, sequence_length, appliance, model_type, epochs)
            model_class.load_weights()
            
            test_loss, test_mae = model_class.model.evaluate_generator(test_generator)
            print('{:40s} MAE: {:.3f}'.format(model_class.name, test_mae*scaler_.std))
            
            list_appliance.append(appliance)
            list_model_name.append('_'.join(model_class.name.split('_')[1: -4]) + '_std_loss')
            list_metric.append('mae')
            list_evaluation.append(test_mae*scaler_.std)

Loading Data for kettle
Time elapse: 23.52s
kettle_jkdae_128to128_reg_std_loss       MAE: 50.626
kettle_jklstm_128to128_reg_std_loss      MAE: 14.957
kettle_s2s_128to128_reg_std_loss         MAE: 31.908
kettle_bi_tcn_residual_128to128_reg_std_loss MAE: 36.866
kettle_fcn_128to128_reg_std_loss         MAE: 17.588
Loading Data for fridge
Time elapse: 70.07s
fridge_jkdae_512to512_reg_std_loss       MAE: 29.689
fridge_jklstm_512to512_reg_std_loss      MAE: 4.609
fridge_s2s_512to512_reg_std_loss         MAE: 9.826
fridge_bi_tcn_residual_512to512_reg_std_loss MAE: 15.134
fridge_fcn_512to512_reg_std_loss         MAE: 5.622
Loading Data for microwave
Time elapse: 38.95s
microwave_jkdae_288to288_reg_std_loss    MAE: 26.190
microwave_jklstm_288to288_reg_std_loss   MAE: 5.480
microwave_s2s_288to288_reg_std_loss      MAE: 25.749
microwave_bi_tcn_residual_288to288_reg_std_loss MAE: 27.347
microwave_fcn_288to288_reg_std_loss      MAE: 6.216
Loading Data for dishwasher
Time elapse: 15.15s
dishwasher_j

In [13]:
# save result
df = pd.DataFrame({
    'appliance' : list_appliance,
    'model': list_model_name,
    'metric': list_metric,
    'evaluation': list_evaluation
})
# fix model name error for washing_machine
df['model'] = df['model'].apply(lambda x: x[8:] if x[:8] == 'machine_' else x)
df.to_csv('evluation_result.csv', index=False)
df

appliance                      model metric  evaluation
0             kettle                      jkdae    mae   50.449544
1             kettle                     jklstm    mae   12.015647
2             kettle                        s2s    mae   28.535561
3             kettle            bi_tcn_residual    mae   35.205554
4             kettle                        fcn    mae   11.591790
5             fridge                      jkdae    mae   29.845080
6             fridge                     jklstm    mae    2.856283
7             fridge                        s2s    mae    8.532750
8             fridge            bi_tcn_residual    mae   13.820671
9             fridge                        fcn    mae    4.401467
10         microwave                      jkdae    mae   23.974915
11         microwave                     jklstm    mae    5.133209
12         microwave                        s2s    mae   22.149577
13         microwave            bi_tcn_residual    mae   24.806346
14         microwave                        fcn    mae    5.744956
15        dishwasher                      jkdae    mae   47.394921
16        dishwasher                     jklstm    mae   10.731595
17        dishwasher                        s2s    mae   28.380430
18        dishwasher            bi_tcn_residual    mae  102.256200
19        dishwasher                        fcn    mae    4.612818
20   washing_machine                      jkdae    mae   48.747192
21   washing_machine                     jklstm    mae   12.879223
22   washing_machine                        s2s    mae   27.796159
23   washing_machine            bi_tcn_residual    mae   63.550280
24   washing_machine                        fcn    mae   10.968467
25            kettle                      jkdae    acc    0.983935
26            kettle                     jklstm    acc    0.993610
27            kettle                        s2s    acc    0.978268
28            kettle            bi_tcn_residual    acc    0.960095
29            kettle                        fcn    acc    0.993247
..               ...                        ...    ...         ...
95   washing_machine            jkdae_std_block    mae  158.844314
96   washing_machine           jklstm_std_block    mae  144.597305
97   washing_machine              s2s_std_block    mae   29.512716
98   washing_machine  bi_tcn_residual_std_block    mae   66.270481
99   washing_machine              fcn_std_block    mae   13.861414
100           kettle             jkdae_std_loss    mae   50.625945
101           kettle            jklstm_std_loss    mae   14.956994
102           kettle               s2s_std_loss    mae   31.907694
103           kettle   bi_tcn_residual_std_loss    mae   36.865718
104           kettle               fcn_std_loss    mae   17.588080
105           fridge             jkdae_std_loss    mae   29.688570
106           fridge            jklstm_std_loss    mae    4.609200
107           fridge               s2s_std_loss    mae    9.826387
108           fridge   bi_tcn_residual_std_loss    mae   15.134424
109           fridge               fcn_std_loss    mae    5.622331
110        microwave             jkdae_std_loss    mae   26.190142
111        microwave            jklstm_std_loss    mae    5.480484
112        microwave               s2s_std_loss    mae   25.749362
113        microwave   bi_tcn_residual_std_loss    mae   27.346793
114        microwave               fcn_std_loss    mae    6.216223
115       dishwasher             jkdae_std_loss    mae   58.322897
116       dishwasher            jklstm_std_loss    mae   36.473134
117       dishwasher               s2s_std_loss    mae   31.893784
118       dishwasher   bi_tcn_residual_std_loss    mae  134.840762
119       dishwasher               fcn_std_loss    mae    8.426154
120  washing_machine             jkdae_std_loss    mae   66.820842
121  washing_machine            jklstm_std_loss    mae   25.467932
122  washing_machine               s2s_std_loss    mae   2